# Overview

This file is for data wrangling of the output data from engineering software into Python Pandas dataframes to then conduct machine learning. 


# Modelica Mat files

Mat files are created from a physics-based simulation of a commercial building HVAC system. The goal is to convert the Mat files into a python pandas dataframe.

First import relevant libraries. 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from buildingspy.io.outputfile import Reader # For I/O on the Mat file
% matplotlib inline

# Small mat files can be directly converted into pandas dataframes

In [1]:
'''
Opens a dymola mat file to extract out selected variables, which is returned as a dataframe. 
Missing data is indicated by NaN

Output: dataframe
Input:
    matfile_name        name of a dymola mat file
    variables           array-like of strings with variables
'''
def Mat2Pickles(matfile_name, variables):
    dymolareader = Reader(matfile_name, "dymola")
    # Get the indices
    timeset = set()
    for variable in variables:
        time, value = dymolareader.values(variable)
        timeset.update(time)
    # Create the dataframe
    df = pd.DataFrame(index=sorted(timeset), columns=variables)
    for variable in variables:
        time, value = dymolareader.values(variable)
        df.loc[time, variable] = value
    return df

# Large Mat files cannot be loaded into memory within Python


An alternative approach is to use a utility

## Create a list of the variable names that are to be exported

In [ ]:
# Open the map of names
var_info = np.genfromtxt('../Data/ReferenceFiles/molecularfoundry_map.csv', delimiter=',', dtype=str)
# The second column contains the dymola names of the variables
variable_names = var_info[:, 1]
# Remove empty entries
variable_names = variable_names[variable_names != '']

## Create a command line command

This command exports the desired variables from the Modelica .mat file into a .csv or .mat file

The command string calls the function alist.exe, which can be found in the /Dymola program folder.

In [4]:
'''
The command is a string that can be created using createlistcmd()
Output:
    cmd_string     string to copy into the command line 
'''
def createalistcmd(variable_names, input_file_name, output_file_name):
    cmd_string = 'alist'
    for variable in variable_names:
        cmd_string += ' -e ' + variable
    cmd_string += ' ' + input_file_name + ' ' + output_file_name
    return cmd_string

In [25]:
createalistcmd(variable_names, 'GP21_efficiencyx75percent.mat', 'GP21_efficiencyx75percent.csv')

'alist -e GP_1.y -e GP2.y -e GP_20.y -e ARU_1.dp2 -e ARU_1.TEvaEnt -e ARU_1.TEvaLvg -e VOut3.V_flow -e ARU_1.TSet -e ARU_1.on -e ARU_1.P -e ARU_1.dp1 -e ARU_1.TConEnt -e ARU_1.TConLvg -e VOut7.V_flow -e ARU_2.dp2 -e ARU_2.TEvaEnt -e ARU_2.TEvaLvg -e VOut2.V_flow -e ARU_2.TSet -e ARU_2.on -e ARU_2.P -e ARU_2.dp1 -e ARU_2.TConEnt -e ARU_2.TConLvg -e VOut6.V_flow -e ARU_3.dp2 -e ARU_3.TEvaEnt -e ARU_3.TEvaLvg -e VOut1.V_flow -e ARU_3.TSet -e ARU_3.on -e ARU_3.P -e ARU_3.dp1 -e ARU_3.TConEnt -e ARU_3.TConLvg -e VOut5.V_flow -e TCHWARU5.T -e TCHWARU4.T -e GP4.N_actual -e secondaryPumpControl.PumpControl -e GP4.P -e TCHWARU6.T -e TCHWARU7.T -e VOut4.V_flow -e senRelPre.port_a.p -e senRelPre.port_b.p -e GP_5.y -e GP6.y -e GP_21.y -e CT01.y -e CT01.TLvg -e CT01.TAppAct -e CT01.PFan -e CT02.y -e CT02.TLvg -e CT02.TAppAct -e CT02.PFan -e weaBus.relHum -e weaBus.TWetBul GP21_efficiencyx75percent.mat GP21_efficiencyx75percent.csv'

## Run the command string

Ensure the following are in the same folder:

    alist.exe which can be found in Programs/Dymola/bin
    
    the dymola.mat file

In the command line, type the alist.exe command created in the following step

After the command finishes executing, the desired .mat or .csv file will be created

# Import CSV files in Python

In [28]:
data = pd.read_csv('test.csv', delimiter=',', header=0, index_col=0)

In [29]:
data.head()

,GP_1.y,GP2.y,GP_20.y,ARU_1.dp2,ARU_1.TEvaEnt,ARU_1.TEvaLvg,VOut3.V_flow,ARU_1.TSet,ARU_1.on,ARU_1.P,...,CT01.y,CT01.TLvg,CT01.TAppAct,CT01.PFan,CT02.y,CT02.TLvg,CT02.TAppAct,CT02.PFan,weaBus.relHum,weaBus.TDryBul
Time,,,,,,,,,,,,,,,,,,,,,
0.000000e+00,0.03,0.03,0.03,0.000000e+00,293.15,278.15,7.395380e-17,280.35,0,0.0,...,1.0,293.150,-0.614284,169987.0,0.0,293.15,11.8195,0.0,0.9,280.35
1.751380e-09,0.03,0.03,0.03,6.984920e-10,293.15,278.15,1.216280e-14,280.35,0,0.0,...,1.0,293.150,-0.614284,169987.0,0.0,293.15,11.8195,0.0,0.9,280.35
1.751380e-09,0.03,0.03,0.03,6.984920e-10,293.15,278.15,1.242100e-14,280.35,0,0.0,...,1.0,293.150,-0.614284,169987.0,0.0,293.15,11.8195,0.0,0.9,280.35
1.000000e+00,0.03,0.03,0.03,1.285440e+00,293.15,278.15,2.242720e-05,280.35,0,0.0,...,1.0,293.148,-0.593598,169987.0,0.0,293.15,11.8221,0.0,0.9,280.35
1.000000e+00,0.03,0.03,0.03,1.285440e+00,293.15,278.15,2.242720e-05,280.35,0,0.0,...,1.0,293.148,-0.593598,169987.0,0.0,293.15,11.8221,0.0,0.9,280.35


# Calculate the variances to remove features that are close to zero

In [10]:
variances = data.var(axis=0)

In [11]:
variances[variances<0.01]

VOut3.V_flow    0.000203
VOut7.V_flow    0.000540
VOut2.V_flow    0.000095
VOut6.V_flow    0.000440
VOut1.V_flow    0.000033
VOut5.V_flow    0.000070
VOut4.V_flow    0.000069
dtype: float64

# Check for duplicates and inconsistent times

In [67]:
'''
One method is to remove the duplicates from the data
'''
def removeDuplicates(df):
    unique_indices = np.unique(df.index, return_index=True)[1]
    return df.iloc[unique_indices]

In [3]:
'''
Replaces duplicate indices with the average
'''
def replaceDuplicatesWithAverage(orig_df):
    # Keep only the 5 minute samples
    df = orig_df[orig_df.index % 300 == 0]
    # Check for identical timestamps and take the average
    df.sort_index(inplace=True)
    start_of_dupes = 0
    for i in range(1, len(df) + 1):
        if i == len(df) or df.index[i] != df.index[i - 1]:
            if i - start_of_dupes > 1:
                average = df.iloc[start_of_dupes:i].mean()
                df.iloc[start_of_dupes] = average
            start_of_dupes = i
    df["___INDEX___"] = df.index
    df.drop_duplicates(subset="___INDEX___", keep="first", inplace=True)
    df.drop("___INDEX___", axis=1, inplace=True)
    return df

# Save the cleaned data as pickle files

In [ ]:
# Convert all the csv files in SimulationResults-smaller-csv to cleaned pandas arrays stored as pickle files
import os
for file in os.listdir("../SimulationResults-smaller-csv"):
    if file.endswith(".csv"):
        temp_pandas = pd.read_csv('../SimulationResults-smaller-csv/' + file, delimiter=',', header=0, index_col=0)
        temp_pandas  = replaceDuplicatesWithAverage(temp_pandas)
        temp_pandas['SecondsOfYear'] = temp_pandas.index # add the seconds of the year as a feature
        newfilename = file.split('.')[0] # remove the .csv extension
        temp_pandas.loc[3900:31539900].to_pickle(newfilename + '.p') # Modify the following